In [1]:
import numpy as np
import random as rand
from sklearn import linear_model, preprocessing
from sklearn.decomposition import PCA
from sklearn.cross_validation import KFold
from scipy.io import loadmat, savemat

In [2]:
def E_abs(y_predict,y):
    E=0
    for i in range(len(y)):
        E+=abs(y_predict[i]-y[i])
    return E/len(y)

def E_01(y_predict,y):
    y_temp=[]
    for i in range(len(y)):
        if y_predict[i]>0.5:
            y_temp.append(1)
        elif y_predict[i]==0.5:
            y_temp.append(rand.randint(0,1))
        else:
            y_temp.append(0)
    return E_abs(y_temp,y)

In [3]:
feat=loadmat("../../data/feat1.mat")

In [4]:
#read training features
len_train=feat["len_train"][0][0]
x=np.concatenate((feat["x1_int"],feat["x1_float"]),axis=1)
x=x[:len_train][:]
dimensions=len(x[0])

In [5]:
#read training truth
y=feat["y"][:len_train]
y=y[:,0]

In [6]:
#training
n_folds=5
iterations=10000
c_list=list(range(-4,5))
c_list[:]=[10 ** x for x in c_list]
n_components_list=list(range(int(dimensions/4),int(dimensions/2)))
count=0
count_limit=300
output=np.empty((feat["len"][0][0]))
for i in range(iterations):
    Ein_01=0
    Ein_abs=0
    Eval_01=0
    Eval_abs=0
    sample_index=np.random.randint(len(x),size=len(x))
    x_sample=x[sample_index,:]
    y_sample=y[sample_index]
    c=rand.choice(c_list)
    n_components=rand.choice(n_components_list)
    kf=KFold(len(x),n_folds=n_folds)
    for train_index, val_index in kf:
        x_train, x_val = x_sample[train_index], x_sample[val_index]
        y_train, y_val = y_sample[train_index], y_sample[val_index]
        pca = PCA(n_components=n_components)
        pca.fit(x_train)
        x_train_reduce = pca.transform(x_train)
        logistic = linear_model.LogisticRegression(C=c)
        logistic.fit(x_train_reduce,y_train)
        y_train_predict = logistic.predict_proba(x_train_reduce)
        y_train_predict = y_train_predict[:,1]
        Ein_01+=E_01(y_train_predict,y_train)
        Ein_abs+=E_abs(y_train_predict,y_train)
        x_val_reduce = pca.transform(x_val)
        y_val_predict = logistic.predict_proba(x_val_reduce)
        y_val_predict = y_val_predict[:,1]
        Eval_01+=E_01(y_val_predict,y_val)
        Eval_abs+=E_abs(y_val_predict,y_val)
    Eval_01/=n_folds
    Eval_abs/=n_folds
    if Eval_01<=0.17 or Eval_abs<=0.32:
        pca = PCA(n_components=n_components)
        pca.fit(x_sample)
        x_reduce = pca.transform(x_sample)
        logistic = linear_model.LogisticRegression(C=c)
        logistic.fit(x_reduce,y_sample)
        x_all=np.concatenate((feat["x1_int"],feat["x1_float"]),axis=1)
        x_all_reduce=pca.transform(x_all)
        y_all_predict = logistic.predict_proba(x_all_reduce)
        y_all_predict = y_all_predict[:,1]
        y_all_predict = np.transpose(y_all_predict)
        output=np.column_stack((output,y_all_predict))
        count+=1
    if count==count_limit:
        break
output=output[:,1:]

In [7]:
savemat("../../result/PCA_logisticReg/blending.mat",{"output":output})